# **Лаб3_Энькаэр Уэркэнь_ ИУ5И-21М**

# **Задание**

1. Выбрать один или несколько наборов данных (датасетов) для решения следующих задач. Каждая задача может быть решена на отдельном датасете, или несколько задач могут быть решены на одном датасете. Просьба не использовать датасет, на котором данная задача решалась в лекции.
2. Для выбранного датасета (датасетов) на основе материалов лекций решить следующие задачи:

  * масштабирование признаков (не менее чем тремя способами);
  * обработку выбросов для числовых признаков (по одному способу для удаления выбросов и для замены выбросов);
  * обработку по крайней мере одного нестандартного признака (который не является числовым или категориальным);
  * отбор признаков:
    * один метод из группы методов фильтрации (filter methods);
    * один метод из группы методов обертывания (wrapper methods);
    * один метод из группы методов вложений (embedded methods).

# **Ход работы**

In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.feature_selection import RFE, SelectKBest, f_classif
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [46]:
# Загрузка датасета
df = pd.read_csv("Tree_Data.csv")
df.head()

,No,Plot,Subplot,Species,Light_ISF,Light_Cat,Core,Soil,Adult,Sterile,...,AMF,EMF,Phenolics,Lignin,NSC,Census,Time,Event,Harvest,Alive
0,126,1,C,Acer saccharum,0.106,Med,2017,Prunus serotina,I,Non-Sterile,...,22.00,NaN,-0.56,13.86,12.15,4,14.0,1.0,NaN,NaN
1,11,1,C,Quercus alba,0.106,Med,2017,Quercus rubra,970,Non-Sterile,...,15.82,31.07,5.19,20.52,19.29,33,115.5,0.0,NaN,X
2,12,1,C,Quercus rubra,0.106,Med,2017,Prunus serotina,J,Non-Sterile,...,24.45,28.19,3.36,24.74,15.01,18,63.0,1.0,NaN,NaN
3,2823,7,D,Acer saccharum,0.080,Med,2016,Prunus serotina,J,Non-Sterile,...,22.23,NaN,-0.71,14.29,12.36,4,14.0,1.0,NaN,NaN
4,5679,14,A,Acer saccharum,0.060,Low,2017,Prunus serotina,689,Non-Sterile,...,21.15,NaN,-0.58,10.85,11.20,4,14.0,1.0,NaN,NaN


# --- Обработка пропусков ---

In [47]:
print("Текущие пропуски:")
print(df.isnull().sum()[df.isnull().sum() > 0])

Текущие пропуски:
EMF        1500
Event         1
Harvest    2079
Alive      2292
dtype: int64


In [48]:
df = df.drop(columns=['Harvest', 'Alive'])  # слишком много пропусков
if 'EMF' in df.columns:
    df['EMF'] = df['EMF'].fillna(df['EMF'].median())
if 'Event' in df.columns:
    df['Event'] = df['Event'].fillna(df['Event'].mode()[0])

Проверка пропусков после обработки

In [49]:
print("Оставшиеся пропуски:")
print(df.isnull().sum()[df.isnull().sum() > 0])

Оставшиеся пропуски:
Series([], dtype: int64)


# --- Масштабирование ---

In [50]:
numeric = df.select_dtypes(include=['float64', 'int64']).drop(columns=['No', 'Core', 'Plot'])
sc_minmax = MinMaxScaler().fit_transform(numeric)
sc_std = StandardScaler().fit_transform(numeric)
sc_robust = RobustScaler().fit_transform(numeric)

In [51]:
print("\nМасштабирование (MinMax):")
print(pd.DataFrame(sc_minmax, columns=numeric.columns).head())

print("\nМасштабирование (Standard):")
print(pd.DataFrame(sc_std, columns=numeric.columns).head())

print("\nМасштабирование (Robust):")
print(pd.DataFrame(sc_robust, columns=numeric.columns).head())


Масштабирование (MinMax):
   Light_ISF     AMF       EMF  Phenolics    Lignin       NSC    Census  \
0   0.573643  0.2200  0.316800   0.106040  0.380812  0.312127  0.000000   
1   0.573643  0.1582  0.355086   0.877852  0.598887  0.596024  1.000000   
2   0.573643  0.2445  0.322171   0.632215  0.737066  0.425845  0.482759   
3   0.372093  0.2223  0.316800   0.085906  0.394892  0.320477  0.000000   
4   0.217054  0.2115  0.316800   0.103356  0.282253  0.274354  0.000000   

       Time  Event  
0  0.000000    1.0  
1  1.000000    0.0  
2  0.482759    1.0  
3  0.000000    1.0  
4  0.000000    1.0  

Масштабирование (Standard):
   Light_ISF       AMF       EMF  Phenolics    Lignin       NSC    Census  \
0   0.791680  0.117566  0.050683  -1.265864 -0.280272 -0.481592 -1.231007   
1   0.791680 -0.384572  0.346915   1.653676  0.702262  1.179840  1.933236   
2   0.791680  0.316634  0.092244   0.724501  1.324829  0.183911  0.296558   
3  -0.222634  0.136254  0.050683  -1.342026 -0.216835 -0.43

# --- Обработка выбросов ---
Удаление выбросов по IQR для NSC

In [52]:
Q1 = df['NSC'].quantile(0.25)
Q3 = df['NSC'].quantile(0.75)
IQR = Q3 - Q1
filtered_df = df[(df['NSC'] >= Q1 - 1.5 * IQR) & (df['NSC'] <= Q3 + 1.5 * IQR)]
print("\nУдаление выбросов: осталось строк:", len(filtered_df))


Удаление выбросов: осталось строк: 2779


Замена выбросов Winsorization для Lignin

In [53]:
df['Lignin'] = stats.mstats.winsorize(df['Lignin'], limits=[0.05, 0.05])

# --- Обработка нестандартного признака (дата) ---

In [55]:
df['PlantDate'] = pd.to_datetime(df['PlantDate'], format="%m/%d/%y", errors='coerce')
df['PlantMonth'] = df['PlantDate'].dt.month
df['PlantDay'] = df['PlantDate'].dt.day

# --- Отбор признаков ---
Target: бинаризуем 'Sterile'

In [56]:
df = df[df['Sterile'].isin(['Sterile', 'Non-Sterile'])]
df['Sterile_bin'] = (df['Sterile'] == 'Non-Sterile').astype(int)

1. Filter method — корреляция

In [57]:
corr = df.corr(numeric_only=True)['Sterile_bin'].drop('Sterile_bin').abs()
filter_top = corr.sort_values(ascending=False).head(5)
print("\nFilter method (корреляция):")
print(filter_top)



Filter method (корреляция):
EMF           0.388015
AMF           0.384148
PlantMonth    0.183937
Core          0.103794
Event         0.077577
Name: Sterile_bin, dtype: float64


Wrapper method — RFE

In [58]:
X = df[numeric.columns].fillna(0)
y = df['Sterile_bin']
model = LogisticRegression(max_iter=1000)
rfe = RFE(model, n_features_to_select=5)
rfe.fit(X, y)
print("\nWrapper method (RFE):")
print(X.columns[rfe.support_])


Wrapper method (RFE):
Index(['Light_ISF', 'AMF', 'EMF', 'Phenolics', 'Lignin'], dtype='object')


Embedded method — Lasso

In [59]:
lasso = LassoCV(cv=5).fit(X, y)
importance = pd.Series(np.abs(lasso.coef_), index=X.columns)
print("\nEmbedded method (Lasso):")
print(importance.sort_values(ascending=False).head(5))


Embedded method (Lasso):
Event     0.080828
AMF       0.010928
EMF       0.010500
Lignin    0.008496
NSC       0.005773
dtype: float64
